In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
df = pd.read_csv('listings.csv.gz')

In [3]:
df2 = df.copy()

In [4]:
df_numeric = df.select_dtypes(include=['float64'])

df_string = df.select_dtypes(include=['object'])

df_int = df.select_dtypes(include=['int'])

In [5]:
potential_important_string = ['host_since', 'host_location', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',\
                             'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed',\
                             'property_type', 'room_type', 'bathrooms_text', 'instant_bookable', 'last_review']

In [6]:
df_string[potential_important_string].describe()

,host_since,host_location,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood,neighbourhood_cleansed,property_type,room_type,bathrooms_text,instant_bookable,last_review
count,18228,18199,3937,7796,18228,18228,12023,18291,18291,18291,18269,18291,15902
unique,3268,534,50,89,2,2,67,22,65,4,27,2,1833
top,2016-04-05,"Amsterdam, North Holland, Netherlands",100%,100%,f,t,"Amsterdam, Noord-Holland, Netherlands",De Baarsjes - Oud-West,Entire apartment,Entire home/apt,1 bath,f,2020-01-02
freq,88,10046,2794,3638,15849,11918,9534,3044,11393,14283,10677,13914,259


In [7]:
drop_list = ['neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated', 'license', 'latitude', 'longitude', 'id',\
             'scrape_id', 'host_id']

In [8]:
df2.drop(columns = drop_list, axis = 1, inplace = True)
df2.shape

(18291, 65)

In [9]:
df2.dropna(subset = ['host_since', 'host_is_superhost', 'host_identity_verified'], inplace = True)

In [10]:
# cleaning and engineering host_since
df2['host_since'] = pd.to_datetime(df2['host_since'])
df2['months_active'] = (dt.datetime.today() - df2['host_since'])/np.timedelta64(1, 'M')
df2['months_active'] = df2['months_active'].astype(int)

In [11]:
df2['host_location'].value_counts()
# too many entries to clean, ignore for now

Amsterdam, North Holland, Netherlands        10046
Amsterdam, North Holland, The Netherlands     3515
NL                                            1549
Amsterdam, Noord-Holland, The Netherlands     1153
Netherlands                                    520
                                             ...  
Athens, Attica, Greece                           1
Jons, Auvergne-Rhône-Alpes, France               1
Bennekom, Gelderland, Netherlands                1
Heerenveen, Friesland, Netherlands               1
Wellington, Wellington, New Zealand              1
Name: host_location, Length: 534, dtype: int64

In [12]:
df2['host_response_rate'] = df2['host_response_rate'].str.replace('%','')
df2['host_response_rate'] = pd.to_numeric(df2['host_response_rate'])
print(df2['host_response_rate'].describe())
# Given most of the values being missing, we will replace missing values with mean
df2['host_response_rate'].fillna(df2['host_response_rate'].mean(), inplace = True)
# print(df2['host_response_rate'].describe())

count    3937.000000
mean       86.042418
std        28.633557
min         0.000000
25%        89.000000
50%       100.000000
75%       100.000000
max       100.000000
Name: host_response_rate, dtype: float64


In [13]:
df2['host_acceptance_rate'] = df2['host_acceptance_rate'].str.replace('%','')
df2['host_acceptance_rate'] = pd.to_numeric(df2['host_acceptance_rate'])
print(df2['host_acceptance_rate'].describe())
# Given most of the values being missing, we will replace missing values with mean
df2['host_acceptance_rate'].fillna(df2['host_acceptance_rate'].mean(), inplace = True)
print(df2['host_acceptance_rate'].describe())

count    7796.000000
mean       77.234864
std        33.575089
min         0.000000
25%        63.000000
50%        98.000000
75%       100.000000
max       100.000000
Name: host_acceptance_rate, dtype: float64
count    18228.000000
mean        77.234864
std         21.956729
min          0.000000
25%         77.234864
50%         77.234864
75%         93.000000
max        100.000000
Name: host_acceptance_rate, dtype: float64


In [14]:
print(df2['host_is_superhost'].value_counts())
# only true false values, convert them to 0 and 1 categorical
df2['host_is_superhost'].replace({'t': 1, 'f': 0}, inplace = True)
df2['host_is_superhost'].value_counts()

f    15849
t     2379
Name: host_is_superhost, dtype: int64


0    15849
1     2379
Name: host_is_superhost, dtype: int64

In [15]:
print(df2['host_identity_verified'].value_counts())
# only true false values, convert them to 0 and 1 categorical
df2['host_identity_verified'].replace({'t': 1, 'f': 0}, inplace = True)
df2['host_identity_verified'].value_counts()

t    11918
f     6310
Name: host_identity_verified, dtype: int64


1    11918
0     6310
Name: host_identity_verified, dtype: int64

In [16]:
df2['neighbourhood'].value_counts()
# too many values, ignore for now

Amsterdam, Noord-Holland, Netherlands             9487
Amsterdam, North Holland, Netherlands             1483
Amsterdam, NH, Netherlands                         697
Amsterdam, Netherlands                             110
Amsterdam-Zuidoost, Noord-Holland, Netherlands      57
                                                  ... 
Amstelveen, NH, Netherlands                          1
Diemen, Noord-Holland, Netherlands                   1
Amsterdam, SN, Netherlands                           1
Haarlem, Noord-Holland, Netherlands                  1
Amsterdam, VX, Netherlands                           1
Name: neighbourhood, Length: 67, dtype: int64

In [17]:
print(df2['neighbourhood_cleansed'].value_counts())
# manageable number of values, will do one hot encoding
neighbourhood_dummies = pd.get_dummies(df2['neighbourhood_cleansed'], prefix = 'Neighbourhood', drop_first=True)
df2_dummies = pd.concat([df2, neighbourhood_dummies], axis=1)
df2_dummies.shape

De Baarsjes - Oud-West                    3029
De Pijp - Rivierenbuurt                   2262
Centrum-West                              1994
Centrum-Oost                              1554
Westerpark                                1392
Zuid                                      1306
Oud-Oost                                  1177
Bos en Lommer                             1042
Oostelijk Havengebied - Indische Buurt     869
Oud-Noord                                  586
Watergraafsmeer                            525
IJburg - Zeeburgereiland                   423
Slotervaart                                410
Noord-West                                 369
Noord-Oost                                 254
Buitenveldert - Zuidas                     252
Geuzenveld - Slotermeer                    214
De Aker - Nieuw Sloten                     126
Osdorp                                     124
Gaasperdam - Driemond                      121
Bijlmer-Centrum                            103
Bijlmer-Oost 

(18228, 87)

In [18]:
df2_dummies['property_type'].value_counts()
#too many values, ignore

Entire apartment                     11335
Private room in apartment             2217
Entire house                          1195
Entire townhouse                       472
Private room in bed and breakfast      344
                                     ...  
Cave                                     1
Room in casa particular                  1
Earth house                              1
Tipi                                     1
Bus                                      1
Name: property_type, Length: 65, dtype: int64

In [19]:
print(df2_dummies['room_type'].value_counts())
# manageable number of values, will do one hot encoding
room_dummies = pd.get_dummies(df2_dummies['room_type'], prefix = 'room_type', drop_first=True)
df2_dummies = pd.concat([df2_dummies, room_dummies], axis=1)
df2_dummies.shape

Entire home/apt    14221
Private room        3830
Hotel room           127
Shared room           50
Name: room_type, dtype: int64


(18228, 90)

In [20]:
print(df2_dummies['bathrooms_text'].value_counts())
# manageable number of values, will do one hot encoding
bathroom_dummies = pd.get_dummies(df2_dummies['bathrooms_text'], prefix = 'bathrooms', drop_first=True)
df2_dummies = pd.concat([df2_dummies, bathroom_dummies], axis=1)
df2_dummies.shape

1 bath               10637
1.5 baths             3452
1 shared bath         1210
1 private bath        1043
2 baths                800
1.5 shared baths       466
2.5 baths              239
3 baths                 96
3.5 baths               43
Half-bath               39
0 baths                 38
Shared half-bath        32
2 shared baths          29
0 shared baths          15
4 baths                 13
Private half-bath       11
3 shared baths          10
3.5 shared baths         8
2.5 shared baths         6
4 shared baths           5
4.5 baths                4
8 baths                  3
5 baths                  2
6 baths                  2
12 baths                 1
13 baths                 1
5.5 baths                1
Name: bathrooms_text, dtype: int64


(18228, 116)

In [21]:
print(df2_dummies['instant_bookable'].value_counts())
# manageable number of values, will do one hot encoding
df2_dummies['instant_bookable'].replace({'t': 1, 'f': 0}, inplace = True)
df2_dummies['instant_bookable'].value_counts()

f    13854
t     4374
Name: instant_bookable, dtype: int64


0    13854
1     4374
Name: instant_bookable, dtype: int64

In [22]:
# df2_dummies['last_review'] = pd.to_datetime(df2_dummies['last_review'])
# df2_dummies['last_review'] = (dt.datetime.today() - df2_dummies['last_review'])/np.timedelta64(1, 'M')
# # df2_dummies['last_review'] = df2_dummies['last_review'].astype(int)

In [23]:
df2_dummies['last_review'] = pd.to_datetime(df2_dummies['last_review'])
print(df2_dummies['last_review'].isnull().sum())
# print(df2_dummies['last_review'].describe())
# there are a lot of null values, so we will not remove them as we will lose data. We can instead replace missing values with
# the most frequent observation which is 2020-01-02 (258/15845 % frequency)
df2_dummies['last_review'].fillna(df2_dummies['last_review'].mode(), inplace = True)
df2['last_review'].describe()

2383


count          15845
unique          1830
top       2020-01-02
freq             258
Name: last_review, dtype: object

In [24]:
df2_dummies['last_review'][df2_dummies['last_review'].isnull() == True]

22      NaT
25      NaT
181     NaT
241     NaT
272     NaT
         ..
18286   NaT
18287   NaT
18288   NaT
18289   NaT
18290   NaT
Name: last_review, Length: 2383, dtype: datetime64[ns]

In [91]:
df2_dummies['last_review'].mode()

0   2020-01-02
dtype: datetime64[ns]